# Data compilation from database scan

The trained models have been used for scaning the whole database looking for incentives.\
Each sentence has been firs classified into one of to categories, *Incentive* or *Not-Incentive*.\
Sentences which were classified as incentives were further classified into one of the six policy instruments.

Thus 

In [1]:
import boto3
import json

In [3]:
path = 'C:/Users/jordi/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [13]:
def sentence_stats(sentence_text, min_sentence_length, max_sentence_length, sum_sentence_length):
    sum_sentence_length = sum_sentence_length + len(sentence_text)
    if len(sentence_text) < min_sentence_length:
        min_sentence_length = len(sentence_text)
    if len(sentence_text) > max_sentence_length:
        max_sentence_length = len(sentence_text)
    return min_sentence_length, max_sentence_length, sum_sentence_length

In [15]:
prefix = "spanish_documents/HSSC/sentences/"

i = 0
documents_with_incentives = 0

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
  if ".json" in obj.key:
    i += 1
#     if i % 10 == 0:
#       output.clear()
    if i < 10:
#         print(f"\n\nFile number: {i}\n\n")
        print(i,  end='\r')
        name = obj.key.split("/")[3].split("_")[0]
        flag = True
        sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
        incentive_counter = 0
        sentences_counter = 0
        min_sentence_length = 100000
        max_sentence_length = 0
        sum_sentence_length = 0
        
        for sentence_id in sentences[name]["sentences"]:
            sentence = sentences[name]["sentences"][sentence_id]["text"]
            min_sentence_length, max_sentence_length, sum_sentence_length = sentence_stats(sentence, min_sentence_length, max_sentence_length, sum_sentence_length)
            incentive =sentences[name]["sentences"][sentence_id]["label"]
            if len(incentive) > 1 and incentive[0] == "Incentive":
                incentive_counter += 1
                print(incentive, "\n", sentence)

['Incentive', 'Direct payment'] 
 M. ING. VICTOR H. GARCIA PACHECO RUBRICA.
['Incentive', 'Supplies'] 
 GERENTE DE INFRAESTRUCTURA ING. DANIEL HERNANDEZ ZAMUDIO RUBRICA.
['Incentive', 'Loan'] 
 A lo anterior se suma la entrega de una boleta de garantía a favor de la Ilustre Municipalidad de Panquehue que equivaldrá al diez por ciento (10%) del proyecto de extracción de áridos, calculado por los metros cúbicos a extraer por proyecto, y su valor consignado en la Ordenanza de Derechos Municipales.
['Incentive', 'Loan'] 
 La vigencia de la boleta mencionada anteriormente será de noventa días posterior a la fecha de término del decreto de autorización.
['Incentive', 'Fine'] 
 De ser autorizada la extracción y teniendo a mano dichos documentos se procederá a redactar el decreto alcaldicio, que será el documento y acto administrativo oficial que formalizará el otorgamiento del permiso.
['Incentive', 'Direct payment'] 
 TÍTULO VIII: DERECHOS A PAGAR Y PROCEDIMIENTOS DE COBRO Artículo 17°- Todo

In [17]:
max_sentence_length

970